In [19]:
get_ipython().ast_node_interactivity = 'all'
import pandas as pd
import numpy
from datetime import datetime as dt

from sklearn.model_selection import train_test_split
import sklearn.metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
get_ipython().ast_node_interactivity = 'all'
import pandas 
import numpy

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import sklearn.metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

from sklearn.ensemble import BaggingClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.neural_network import MLPClassifier 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold


In [20]:
from sklearn.neural_network import MLPRegressor

In [21]:
df= pd.read_csv( "final_ml_data.csv")

In [22]:

df['articleDate'] = pd.to_datetime(df['articleDate'])
df['day_of_week'] = df['articleDate'].dt.day_name()
df['month'] = df['articleDate'].dt.month
df['year'] = df['articleDate'].dt.year

In [23]:
df['percentchange_marketcap'] = ((df['close_marketcap']/df['open_marketcap']) - 1) * 100

In [24]:
def replace_source(source):
    if df['source'].value_counts()[source] <= 50:
        return 'other'
    else:
        return source

# Apply the function to the 'source' column
df['source'] = df['source'].apply(replace_source)

In [31]:
df["weekly_change"] = df["open"] - df["close"]
df["up_down"] = ["up" if change >= 0 else "down" for change in df["weekly_change"]]
df['articleDate'] = pd.to_datetime(df['articleDate'])
df['percent_change'] = ((df['close']/df['open']) - 1) * 100
df['invest'] = df.apply(lambda row: 'invest' if (row['close'] / row['open'] - 1) >= 0.01 else 'no invest', axis=1)
df['day_of_week'] = df['articleDate'].dt.day_name()

# Assuming df is your DataFrame
# Define a function to check the count and replace with 'other' if count is 1
def replace_source(source):
    if df['source'].value_counts()[source] <= 50:
        return 'other'
    else:
        return source

# Apply the function to the 'source' column
df['source'] = df['source'].apply(replace_source)

In [32]:
df.dtypes

articleDate                       datetime64[ns]
n_companies                                int64
StockSymbol                               object
source                                    object
companies                                 object
entities                                  object
positive_sentiment                       float64
negative_sentiment                       float64
neutral_sentiment                        float64
title_polarity                           float64
title_subjectivity                       float64
open                                     float64
close                                    float64
high                                     float64
low                                      float64
volume_start                             float64
volume_end                               float64
volume_mean                              float64
country_isUS                               int64
open_marketcap                           float64
close_marketcap     

In [48]:
#Regression
X_train, X_test, y_train, y_test = train_test_split(df, df.percentchange_marketcap, test_size = .5)
lstNum = ["n_companies", "positive_sentiment", "negative_sentiment", "neutral_sentiment", "title_polarity", "title_subjectivity", "open_marketcap"]
lstCat = ["StockSymbol", "source", "day_of_week", "country_isUS"]

pipeNum = Pipeline( [
('selector', ColumnTransformer([ ('selector', 'passthrough', lstNum) ] ) ),
('scaler', StandardScaler() )
])
pipeCat = Pipeline([
('selector', ColumnTransformer([ ('selector', 'passthrough', lstCat) ] )),
('encoder', OneHotEncoder( dtype=int, drop='first', sparse_output=False ) )
])
preprocessor = FeatureUnion([
('cat', pipeCat ),
('num', pipeNum )
])

#from sklearn.neural_network import MLPRegressor

#pipeNnTune = Pipeline( [
#('preprocessor', preprocessor ),
#('model', MLPRegressor( hidden_layer_sizes = (15,15,15), activation = 'relu', max_iter = 5000, learning_rate = 'adaptive') ) 
#    ])


#pipeNnTune.fit( X_train, y_train )
#predTrainNnTune = pipeNnTune.predict( X_train )
#predTestNnTune = pipeNnTune.predict( X_test )

#sklearn.metrics.r2_score( y_train, predTrainNnTune )



In [10]:
 "month", "year"

('month', 'year')

In [26]:
from sklearn.neural_network import MLPRegressor

pipeNnTune = Pipeline( [
('preprocessor', preprocessor ),
('model', MLPRegressor( max_iter = 5000, activation = 'identity', hidden_layer_sizes = (5,5,5)) ) 
    ])


pipeNnTune.fit( X_train, y_train )
predTrainNnTune = pipeNnTune.predict( X_train )
predTestNnTune = pipeNnTune.predict( X_test )

sklearn.metrics.r2_score( y_train, predTrainNnTune )
sklearn.metrics.r2_score( y_test, predTestNnTune )

Pipeline(steps=[('preprocessor',
                 FeatureUnion(transformer_list=[('cat',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['StockSymbol',
                                                                                                    'source',
                                                                                                    'day_of_week',
                                                                                                    'country_isUS'])])),
                                                                 ('encoder',
                                                                  OneHotEncoder(drop='first',
                                                                                dtype=<class 'int'>,
                                                                                sparse_output=False))])),
                                                ('num',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['n_companies',
                                                                                                    'positive_sentiment',
                                                                                                    'negative_sentiment',
                                                                                                    'neutral_sentiment',
                                                                                                    'title_polarity',
                                                                                                    'title_subjectivity',
                                                                                                    'open_marketcap'])])),
                                                                 ('scaler',
                                                                  StandardScaler())]))])),
                ('model',
                 MLPRegressor(activation='identity',
                              hidden_layer_sizes=(5, 5, 5), max_iter=5000))])

0.13418962845548232

0.1166025796688741

In [12]:
sklearn.metrics.r2_score( y_train, predTrainNnTune )

0.13419385364846959

In [13]:
paramGridMLP =  {'hidden_layer_sizes': [ (5,5),(10,10,10),(20,20,20),(40,40),(60,60),(30,30,30) ],
'activation': ['identity', 'logistic', 'tanh', 'relu']} 



In [77]:
numpy.random.seed(28)


In [78]:
#optimal model this is the model we ended up using s our final model however, the case could be made for choosing any one of these models so we kept them in
pipeNnOptimal1 = Pipeline( [
('preprocessor', preprocessor ),
('model', MLPRegressor( max_iter = 5000, activation = 'relu', hidden_layer_sizes = (10,10,10)) ) 
    ])


pipeNnOptimal1.fit( X_train, y_train )
predTrainNnOptimal1 = pipeNnTune.predict( X_train )
predTestNnOptimal1 = pipeNnTune.predict( X_test )

sklearn.metrics.r2_score( y_train, predTrainNnOptimal1 )
sklearn.metrics.r2_score( y_test, predTestNnOptimal1 )

Pipeline(steps=[('preprocessor',
                 FeatureUnion(transformer_list=[('cat',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['StockSymbol',
                                                                                                    'source',
                                                                                                    'day_of_week',
                                                                                                    'country_isUS'])])),
                                                                 ('encoder',
                                                                  OneHotEncoder(drop='first',
                                                                                dtype=<class 'int'>,
                                                                                sparse_output=False))])),
                                                ('num',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['n_companies',
                                                                                                    'positive_sentiment',
                                                                                                    'negative_sentiment',
                                                                                                    'neutral_sentiment',
                                                                                                    'title_polarity',
                                                                                                    'title_subjectivity',
                                                                                                    'open_marketcap'])])),
                                                                 ('scaler',
                                                                  StandardScaler())]))])),
                ('model',
                 MLPRegressor(hidden_layer_sizes=(10, 10, 10), max_iter=5000))])

0.12919438731431365

0.12105020353424722

In [79]:
df['Predicted_percent_change_in_MC'] = pipeNnOptimal1.predict(df)



In [80]:
X_train, X_test, y_train, y_test = train_test_split(df, df.invest, test_size = .5)
lstNum = ["n_companies", "positive_sentiment", "negative_sentiment", "neutral_sentiment", "title_polarity", "title_subjectivity", "open", "volume_start", "Predicted_percent_change_in_MC"]
lstCat = ["StockSymbol", "source", "day_of_week"]

pipeNum = Pipeline( [
('selector', ColumnTransformer([ ('selector', 'passthrough', lstNum) ] ) ),
('scaler', StandardScaler() )
])
pipeCat = Pipeline([
('selector', ColumnTransformer([ ('selector', 'passthrough', lstCat) ] )),
('encoder', OneHotEncoder( dtype=int, drop='first', sparse_output=False ) )
])
preprocessor = FeatureUnion([
('cat', pipeCat ),
('num', pipeNum )
])

from sklearn.ensemble import GradientBoostingClassifier
pipeGbA = Pipeline([
('preprocessor', preprocessor ),
('model', GradientBoostingClassifier() )
], verbose=True )

pipeGbA.fit( X_train, y_train )
predTrain = pipe.predict( X_train )
predTest = pipe.predict( X_test )

sklearn.metrics.balanced_accuracy_score( y_train, predTrain )
sklearn.metrics.balanced_accuracy_score( y_test, predTest )

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   3.6s


Pipeline(steps=[('preprocessor',
                 FeatureUnion(transformer_list=[('cat',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['StockSymbol',
                                                                                                    'source',
                                                                                                    'day_of_week'])])),
                                                                 ('encoder',
                                                                  OneHotEncoder(drop='first',
                                                                                dtype=<class 'int'>,
                                                                                sparse_output=False))])),
                                                ('num',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['n_companies',
                                                                                                    'positive_sentiment',
                                                                                                    'negative_sentiment',
                                                                                                    'neutral_sentiment',
                                                                                                    'title_polarity',
                                                                                                    'title_subjectivity',
                                                                                                    'open',
                                                                                                    'volume_start',
                                                                                                    'Predicted_percent_change_in_MC'])])),
                                                                 ('scaler',
                                                                  StandardScaler())]))])),
                ('model', GradientBoostingClassifier())],
         verbose=True)

0.9959865370349497

0.9957524105134564

In [81]:
X_train, X_test, y_train, y_test = train_test_split(df, df.percentchange_marketcap, test_size = .5)
lstNum = ["n_companies", "positive_sentiment", "negative_sentiment", "neutral_sentiment", "title_polarity", "title_subjectivity", "open_marketcap"]
lstCat = ["StockSymbol", "source", "day_of_week", "country_isUS"]

pipeNum = Pipeline( [
('selector', ColumnTransformer([ ('selector', 'passthrough', lstNum) ] ) ),
('scaler', StandardScaler() )
])
pipeCat = Pipeline([
('selector', ColumnTransformer([ ('selector', 'passthrough', lstCat) ] )),
('encoder', OneHotEncoder( dtype=int, drop='first', sparse_output=False ) )
])
preprocessor = FeatureUnion([
('cat', pipeCat ),
('num', pipeNum )
])

In [82]:
#second best model
pipeNnOptimal2 = Pipeline( [
('preprocessor', preprocessor ),
('model', MLPRegressor( max_iter = 5000, activation = 'tanh', hidden_layer_sizes = (20,20,20)) ) 
    ])


pipeNnOptimal2.fit( X_train, y_train )
predTrainNnOptimal2 = pipeNnOptimal2.predict( X_train )
predTestNnOptimal2 = pipeNnOptimal2.predict( X_test )

sklearn.metrics.r2_score( y_train, predTrainNnOptimal2 )
sklearn.metrics.r2_score( y_test, predTestNnOptimal2 )

Pipeline(steps=[('preprocessor',
                 FeatureUnion(transformer_list=[('cat',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['StockSymbol',
                                                                                                    'source',
                                                                                                    'day_of_week',
                                                                                                    'country_isUS'])])),
                                                                 ('encoder',
                                                                  OneHotEncoder(drop='first',
                                                                                dtype=<class 'int'>,
                                                                                sparse_output=False))])),
                                                ('num',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['n_companies',
                                                                                                    'positive_sentiment',
                                                                                                    'negative_sentiment',
                                                                                                    'neutral_sentiment',
                                                                                                    'title_polarity',
                                                                                                    'title_subjectivity',
                                                                                                    'open_marketcap'])])),
                                                                 ('scaler',
                                                                  StandardScaler())]))])),
                ('model',
                 MLPRegressor(activation='tanh',
                              hidden_layer_sizes=(20, 20, 20),
                              max_iter=5000))])

0.6098410105818708

0.13192775098181897

In [83]:
df['Predicted_percent_change_in_MC'] = pipeNnOptimal2.predict(df)



In [84]:
X_train, X_test, y_train, y_test = train_test_split(df, df.invest, test_size = .5)
lstNum = ["n_companies", "positive_sentiment", "negative_sentiment", "neutral_sentiment", "title_polarity", "title_subjectivity", "open", "volume_start", "Predicted_percent_change_in_MC"]
lstCat = ["StockSymbol", "source", "day_of_week"]

pipeNum = Pipeline( [
('selector', ColumnTransformer([ ('selector', 'passthrough', lstNum) ] ) ),
('scaler', StandardScaler() )
])
pipeCat = Pipeline([
('selector', ColumnTransformer([ ('selector', 'passthrough', lstCat) ] )),
('encoder', OneHotEncoder( dtype=int, drop='first', sparse_output=False ) )
])
preprocessor = FeatureUnion([
('cat', pipeCat ),
('num', pipeNum )
])

from sklearn.ensemble import GradientBoostingClassifier
pipeGbA = Pipeline([
('preprocessor', preprocessor ),
('model', GradientBoostingClassifier() )
], verbose=True )

pipeGbA.fit( X_train, y_train )
predTrain = pipe.predict( X_train )
predTest = pipe.predict( X_test )

sklearn.metrics.balanced_accuracy_score( y_train, predTrain )
sklearn.metrics.balanced_accuracy_score( y_test, predTest )

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   3.4s


Pipeline(steps=[('preprocessor',
                 FeatureUnion(transformer_list=[('cat',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['StockSymbol',
                                                                                                    'source',
                                                                                                    'day_of_week'])])),
                                                                 ('encoder',
                                                                  OneHotEncoder(drop='first',
                                                                                dtype=<class 'int'>,
                                                                                sparse_output=False))])),
                                                ('num',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['n_companies',
                                                                                                    'positive_sentiment',
                                                                                                    'negative_sentiment',
                                                                                                    'neutral_sentiment',
                                                                                                    'title_polarity',
                                                                                                    'title_subjectivity',
                                                                                                    'open',
                                                                                                    'volume_start',
                                                                                                    'Predicted_percent_change_in_MC'])])),
                                                                 ('scaler',
                                                                  StandardScaler())]))])),
                ('model', GradientBoostingClassifier())],
         verbose=True)

0.9955343053218204

0.9954198221144054

In [85]:
X_train, X_test, y_train, y_test = train_test_split(df, df.percentchange_marketcap, test_size = .5)
lstNum = ["n_companies", "positive_sentiment", "negative_sentiment", "neutral_sentiment", "title_polarity", "title_subjectivity", "open_marketcap"]
lstCat = ["StockSymbol", "source", "day_of_week", "country_isUS"]

pipeNum = Pipeline( [
('selector', ColumnTransformer([ ('selector', 'passthrough', lstNum) ] ) ),
('scaler', StandardScaler() )
])
pipeCat = Pipeline([
('selector', ColumnTransformer([ ('selector', 'passthrough', lstCat) ] )),
('encoder', OneHotEncoder( dtype=int, drop='first', sparse_output=False ) )
])
preprocessor = FeatureUnion([
('cat', pipeCat ),
('num', pipeNum )
])

In [86]:
#third best model
pipeNnOptimal3 = Pipeline( [
('preprocessor', preprocessor ),
('model', MLPRegressor( max_iter = 5000, activation = 'logistic', hidden_layer_sizes = (20,20,20)) ) 
    ])


pipeNnOptimal3.fit( X_train, y_train )
predTrainNnOptimal3 = pipeNnOptimal3.predict( X_train )
predTestNnOptimal3 = pipeNnOptimal3.predict( X_test )

sklearn.metrics.r2_score( y_train, predTrainNnOptimal3 )
sklearn.metrics.r2_score( y_test, predTestNnOptimal3 )

Pipeline(steps=[('preprocessor',
                 FeatureUnion(transformer_list=[('cat',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['StockSymbol',
                                                                                                    'source',
                                                                                                    'day_of_week',
                                                                                                    'country_isUS'])])),
                                                                 ('encoder',
                                                                  OneHotEncoder(drop='first',
                                                                                dtype=<class 'int'>,
                                                                                sparse_output=False))])),
                                                ('num',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['n_companies',
                                                                                                    'positive_sentiment',
                                                                                                    'negative_sentiment',
                                                                                                    'neutral_sentiment',
                                                                                                    'title_polarity',
                                                                                                    'title_subjectivity',
                                                                                                    'open_marketcap'])])),
                                                                 ('scaler',
                                                                  StandardScaler())]))])),
                ('model',
                 MLPRegressor(activation='logistic',
                              hidden_layer_sizes=(20, 20, 20),
                              max_iter=5000))])

0.36657719140667433

0.1632815221889331

In [87]:
df['Predicted_percent_change_in_MC'] = pipeNnOptimal3.predict(df)

In [88]:
X_train, X_test, y_train, y_test = train_test_split(df, df.invest, test_size = .5)
lstNum = ["n_companies", "positive_sentiment", "negative_sentiment", "neutral_sentiment", "title_polarity", "title_subjectivity", "open", "volume_start", "Predicted_percent_change_in_MC"]
lstCat = ["StockSymbol", "source", "day_of_week"]

pipeNum = Pipeline( [
('selector', ColumnTransformer([ ('selector', 'passthrough', lstNum) ] ) ),
('scaler', StandardScaler() )
])
pipeCat = Pipeline([
('selector', ColumnTransformer([ ('selector', 'passthrough', lstCat) ] )),
('encoder', OneHotEncoder( dtype=int, drop='first', sparse_output=False ) )
])
preprocessor = FeatureUnion([
('cat', pipeCat ),
('num', pipeNum )
])

from sklearn.ensemble import GradientBoostingClassifier
pipeGbA = Pipeline([
('preprocessor', preprocessor ),
('model', GradientBoostingClassifier() )
], verbose=True )

pipeGbA.fit( X_train, y_train )
predTrain = pipe.predict( X_train )
predTest = pipe.predict( X_test )

sklearn.metrics.balanced_accuracy_score( y_train, predTrain )
sklearn.metrics.balanced_accuracy_score( y_test, predTest )

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   3.4s


Pipeline(steps=[('preprocessor',
                 FeatureUnion(transformer_list=[('cat',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['StockSymbol',
                                                                                                    'source',
                                                                                                    'day_of_week'])])),
                                                                 ('encoder',
                                                                  OneHotEncoder(drop='first',
                                                                                dtype=<class 'int'>,
                                                                                sparse_output=False))])),
                                                ('num',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['n_companies',
                                                                                                    'positive_sentiment',
                                                                                                    'negative_sentiment',
                                                                                                    'neutral_sentiment',
                                                                                                    'title_polarity',
                                                                                                    'title_subjectivity',
                                                                                                    'open',
                                                                                                    'volume_start',
                                                                                                    'Predicted_percent_change_in_MC'])])),
                                                                 ('scaler',
                                                                  StandardScaler())]))])),
                ('model', GradientBoostingClassifier())],
         verbose=True)

0.9947370628417449

0.995209470315076